In [1]:
from PIL import Image
import os
# https://pillow.readthedocs.io/en/stable/handbook/tutorial.html

In [2]:
ROOT_OF_IMAGES = './ppcovers/'
REGION_FOLDERS = ["africa","oceania","asia","south_america","north_america","europe"]


region_to_full_pp_cover_paths = {}
for region_name in REGION_FOLDERS:
  region_folder_path = os.path.join(ROOT_OF_IMAGES,region_name)
  pp_images_in_folder = os.listdir(region_folder_path)
  full_image_paths_for_region = [os.path.join(region_folder_path,pp_image_filename) for pp_image_filename in pp_images_in_folder]
  region_to_full_pp_cover_paths[region_name] = full_image_paths_for_region
region_to_full_pp_cover_paths


{'africa': ['./ppcovers/africa/Kenya.png',
  './ppcovers/africa/Ghana.png',
  './ppcovers/africa/Liberia.png',
  './ppcovers/africa/Sao+Tome+and+Principe.png',
  './ppcovers/africa/Namibia.png',
  './ppcovers/africa/Tanzania.png',
  './ppcovers/africa/Zambia.png',
  './ppcovers/africa/Djibouti.png',
  './ppcovers/africa/Angola.png',
  './ppcovers/africa/Lesotho.png',
  './ppcovers/africa/Tunisia.png',
  './ppcovers/africa/Sierra+Leone.png',
  './ppcovers/africa/Guinea.png',
  './ppcovers/africa/Algeria.png',
  './ppcovers/africa/Rwanda.png',
  './ppcovers/africa/Cameroon.png',
  './ppcovers/africa/Equatorial+Guinea.png',
  './ppcovers/africa/Burundi.png',
  './ppcovers/africa/Seychelles.png',
  './ppcovers/africa/Mozambique.png',
  './ppcovers/africa/Benin.png',
  './ppcovers/africa/Gabon.png',
  './ppcovers/africa/Libya.png',
  './ppcovers/africa/Cape+Verde.png',
  './ppcovers/africa/South+Sudan.png',
  './ppcovers/africa/Madagascar.png',
  './ppcovers/africa/Senegal.png',
  './ppcove

In [3]:
from collections import defaultdict

image_sizes = defaultdict(lambda :  0)
image_aspect_ratios = defaultdict(lambda :  0)
for region in region_to_full_pp_cover_paths:
  for image_path in region_to_full_pp_cover_paths[region]:
    image = Image.open(image_path)
    image_sizes[image.size] += 1
    image_aspect_ratios[image.size] += 1


for image_size in image_sizes:
  print(f"{image_size}:{image_sizes[image_size]}")
# print(image_aspect_ratios)
print(f"min image size: {min([size[0] for size in image_sizes.keys()])} ")


(667, 1000):32
(674, 1000):68
(705, 1000):54
(327, 485):8
(1056, 1500):1
(598, 850):1
(323, 485):2
(308, 448):2
(707, 1000):17
(723, 1000):1
(719, 1000):7
(718, 1000):1
(710, 1000):1
(561, 764):1
(696, 1000):1
(667, 971):1
(1087, 1542):1
min image size: 308 


The region is defined by a 4-tuple, where coordinates are (left, upper, right, lower). The Python Imaging Library uses a coordinate system with (0, 0) in the upper left corner. Also note that coordinates refer to positions between the pixels, so the region in the above example is exactly 300x300 pixels.

In [4]:
# box should be (667 wide, and 10000 tall)

def get_box_for_image_path(image_path):
  desired_box_width = 667
  desired_box_height = 1000   # 2/3 aspect ratio
  image = Image.open(image_path)
  (image_width, image_height) = image.size
  extra_width_pixels = image_width - desired_box_width
  extra_height_pixels = image_height - desired_box_height
  from_left_box_start = int(extra_width_pixels / 2)
  right_end = from_left_box_start + desired_box_width
  from_top_box_start = int(extra_height_pixels / 2)
  bottom_end = from_top_box_start + desired_box_height  
  
  box = (from_left_box_start, from_top_box_start, right_end, bottom_end)
  print(f"Box is {box}")
  return box



# for region in region_to_full_pp_cover_paths:
#   for image_path in region_to_full_pp_cover_paths[region]:

# swiss_reisenpass = Image.open("./ppcovers/europe/Switzerland.png")
# swiss_reisenpass.show()
# box = get_box_for_image_path("./ppcovers/europe/Switzerland.png")
# resize = swiss_reisenpass.resize((705, 1000))
# print(f"Region shape is {resize.size}")
# resize.show()

In [ ]:
def merge_horizontally(im1, im2, spacing=0, bg_color=None):
    """Merge two images horizontally with optional spacing between them"""
    if bg_color is None:
        bg_color = (250, 250, 250, 255)  # Default background color
    
    w = im1.size[0] + spacing + im2.size[0]
    h = max(im1.size[1], im2.size[1])
    im = Image.new("RGBA", (w, h), bg_color)

    # Use alpha channel as mask to prevent black artifacts
    if im1.mode == 'RGBA':
        im.paste(im1, (0, 0), im1)
    else:
        im.paste(im1, (0, 0))

    if im2.mode == 'RGBA':
        im.paste(im2, (im1.size[0] + spacing, 0), im2)
    else:
        im.paste(im2, (im1.size[0] + spacing, 0))

    return im

def merge_vertically(im1, im2, spacing=0, bg_color=None, center=False):
    """Merge two images vertically with optional spacing between them"""
    if bg_color is None:
        bg_color = (250, 250, 250, 255)  # Default background color
    
    h = im1.size[1] + spacing + im2.size[1]
    w = max(im1.size[0], im2.size[0])
    im = Image.new("RGBA", (w, h), bg_color)

    # Use alpha channel as mask to prevent black artifacts
    if center: 
        smaller_image_width = min(im1.size[0], im2.size[0])
    
    if im1.mode == 'RGBA':
        im.paste(im1, (0, 0), im1)
    else:
        im.paste(im1, (0, 0))

    if im2.mode == 'RGBA':
        im.paste(im2, (0, im1.size[1] + spacing), im2)
    else:
        im.paste(im2, (0, im1.size[1] + spacing))

    return im

def get_processed_image_from_path(image_path,remove_top_left=False):
    pp_image = Image.open(image_path)
    bottom_crop_length = 25
    bar_width = 5

    box = (bar_width,bar_width,705 - bottom_crop_length, 1000 - bottom_crop_length)
    resized_image = pp_image.resize((705, 1000))
    resized_image = resized_image.crop(box)

    return resized_image

In [ ]:
from PIL import ImageDraw, ImageFont

# Configuration for text labels
DEFAULT_FONT_SIZE = 40
DEFAULT_FONT_FAMILY = "Arial"
TEXT_PADDING = 10
TEXT_BOTTOM_MARGIN = 15  # Extra bottom margin to prevent text cutoff
TEXT_BACKGROUND_COLOR = (255, 255, 255, 255)  # White background
TEXT_COLOR = (0, 0, 0, 255)  # Black text

# Configuration for poster title
DEFAULT_TITLE_FONT_SIZE = 120
DEFAULT_TITLE_FONT_FAMILY = "Helvetica-Bold"
DEFAULT_TITLE_HEIGHT = 200  # Height of the title row in pixels
TITLE_BACKGROUND_COLOR = (255, 255, 255, 255)  # White background
TITLE_TEXT_COLOR = (0, 0, 0, 255)  # Black text

# Configuration for layout spacing
DEFAULT_HORIZONTAL_SPACING = 0  # Horizontal spacing between passport images in pixels
DEFAULT_VERTICAL_SPACING = 0  # Vertical spacing between rows in pixels

# Configuration for poster background
DEFAULT_BACKGROUND_COLOR = (250, 250, 250, 255)  # Very light gray for poster background

# Configuration for poster margins
DEFAULT_LEFT_MARGIN = 5  # Left margin in pixels
DEFAULT_RIGHT_MARGIN = 5  # Right margin in pixels

# Configuration for footer
DEFAULT_FOOTER_FONT_SIZE = 30
DEFAULT_FOOTER_FONT_FAMILY = "Arial"
DEFAULT_FOOTER_HEIGHT = 100  # Height of the footer row in pixels
FOOTER_BACKGROUND_COLOR = (255, 255, 255, 255)  # White background
FOOTER_TEXT_COLOR = (0, 0, 0, 255)  # Black text
FOOTER_LEFT_MARGIN = 40  # Left margin for footer text in pixels

def format_country_name(country_name):
    """Convert file-friendly name to display name"""
    return country_name.replace("+", " ")

def add_text_label_to_image(image, text, font_size=DEFAULT_FONT_SIZE, font_family=DEFAULT_FONT_FAMILY, 
                            text_color=TEXT_COLOR, bg_color=TEXT_BACKGROUND_COLOR):
    """Add a text label below the passport image"""
    # Reduce font size for long country names (> 22 characters)
    adjusted_font_size = font_size
    if len(text) > 24:
        adjusted_font_size = int(font_size * 0.74)  # Reduce to 74% of original size
    
    # Try to load the BOLD version of the specified font
    font = None
    bold_font_variations = [
        f"{font_family}-Bold",  # Try font name with -Bold suffix
        f"{font_family} Bold",  # Try font name with space Bold
        font_family,  # Try as-is (might already be bold)
        f"/System/Library/Fonts/Supplemental/{font_family} Bold.ttf",
        f"/System/Library/Fonts/Supplemental/{font_family}-Bold.ttf",
        f"/System/Library/Fonts/Supplemental/{font_family}.ttf",
        "/System/Library/Fonts/Supplemental/Arial Bold.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]

    for font_variant in bold_font_variations:
        try:
            font = ImageFont.truetype(font_variant, adjusted_font_size)
            break  # Success! Stop trying
        except:
            continue

    if font is None:
        # Use default PIL font as last resort
        font = ImageFont.load_default()
        print(f"Warning: Could not load bold font for '{font_family}', using default font")
    
    # Calculate text size
    draw_temp = ImageDraw.Draw(Image.new("RGBA", (1, 1)))
    text_bbox = draw_temp.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    
    # Create new image with space for text
    img_width, img_height = image.size
    text_area_height = text_height + TEXT_PADDING + TEXT_BOTTOM_MARGIN
    new_height = img_height + text_area_height
    
    # Create new image with background for text area
    new_image = Image.new("RGBA", (img_width, new_height), bg_color)
    
    # Paste the original passport image at the top (use alpha channel as mask)
    if image.mode == 'RGBA':
        new_image.paste(image, (0, 0), image)
    else:
        new_image.paste(image, (0, 0))
    
    # Draw the text centered below the image
    draw = ImageDraw.Draw(new_image)
    text_x = (img_width - text_width) // 2
    text_y = img_height + TEXT_PADDING
    draw.text((text_x, text_y), text, font=font, fill=text_color)
    
    return new_image

def create_title_row(width, title, title_height=DEFAULT_TITLE_HEIGHT, title_font_size=DEFAULT_TITLE_FONT_SIZE,
                     title_font_family=DEFAULT_TITLE_FONT_FAMILY, title_text_color=TITLE_TEXT_COLOR,
                     title_bg_color=TITLE_BACKGROUND_COLOR):
    """Create a title row image to place at the top of the poster"""
    # Create the title image
    title_image = Image.new("RGBA", (width, title_height), title_bg_color)
    
    # Try to load the specified font
    try:
        font = ImageFont.truetype(title_font_family, title_font_size)
    except:
        try:
            font = ImageFont.truetype(f"/System/Library/Fonts/Supplemental/{title_font_family}.ttf", title_font_size)
        except:
            try:
                base_font = title_font_family.replace("-Bold", "")
                font = ImageFont.truetype(f"/System/Library/Fonts/Supplemental/{base_font}.ttf", title_font_size)
            except:
                try:
                    font = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", title_font_size)
                except:
                    font = ImageFont.load_default()
                    print(f"Warning: Could not load title font '{title_font_family}', using default font")
    
    # Calculate text size and position
    draw = ImageDraw.Draw(title_image)
    text_bbox = draw.textbbox((0, 0), title, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    
    # Center the text both horizontally and vertically
    text_x = (width - text_width) // 2
    text_y = (title_height - text_height) // 2
    
    # Draw the title text
    draw.text((text_x, text_y), title, font=font, fill=title_text_color)
    
    return title_image

def create_footer_row(width, footer_text, footer_height=DEFAULT_FOOTER_HEIGHT, footer_font_size=DEFAULT_FOOTER_FONT_SIZE,
                      footer_font_family=DEFAULT_FOOTER_FONT_FAMILY, footer_text_color=FOOTER_TEXT_COLOR,
                      footer_bg_color=FOOTER_BACKGROUND_COLOR, footer_left_margin=FOOTER_LEFT_MARGIN):
    """Create a footer row image to place at the bottom of the poster"""
    # Create the footer image
    footer_image = Image.new("RGBA", (width, footer_height), footer_bg_color)
    
    # Try to load the specified font with fallback strategies
    font = None
    font_variations = [
        footer_font_family,  # Try as-is first
        f"/System/Library/Fonts/Supplemental/{footer_font_family}.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",  # Final fallback
    ]
    
    for font_path in font_variations:
        try:
            font = ImageFont.truetype(font_path, footer_font_size)
            break  # Success! Stop trying
        except:
            continue
    
    if font is None:
        # Use default font as last resort
        font = ImageFont.load_default()
        print(f"Warning: Could not load footer font '{footer_font_family}', using default font")
    
    # Calculate text size and position
    draw = ImageDraw.Draw(footer_image)
    text_bbox = draw.textbbox((0, 0), footer_text, font=font)
    text_height = text_bbox[3] - text_bbox[1]
    
    # Position text on the left with vertical centering
    text_x = footer_left_margin
    text_y = (footer_height - text_height) // 2
    
    # Draw the footer text
    draw.text((text_x, text_y), footer_text, font=font, fill=footer_text_color)
    
    return footer_image

In [ ]:
def get_country_name_from_path(path):
  country_name = path.split("/")[-1][0:-4]
  return country_name

def get_poster(images_per_row, image_paths, add_labels=True, font_size=DEFAULT_FONT_SIZE, 
               font_family=DEFAULT_FONT_FAMILY, text_color=TEXT_COLOR, bg_color=TEXT_BACKGROUND_COLOR,
               title=None, title_height=DEFAULT_TITLE_HEIGHT, title_font_size=DEFAULT_TITLE_FONT_SIZE,
               title_font_family=DEFAULT_TITLE_FONT_FAMILY, title_text_color=TITLE_TEXT_COLOR,
               title_bg_color=TITLE_BACKGROUND_COLOR, horizontal_spacing=DEFAULT_HORIZONTAL_SPACING,
               vertical_spacing=DEFAULT_VERTICAL_SPACING, background_color=DEFAULT_BACKGROUND_COLOR,
               footer_text=None, footer_height=DEFAULT_FOOTER_HEIGHT, footer_font_size=DEFAULT_FOOTER_FONT_SIZE,
               footer_font_family=DEFAULT_FOOTER_FONT_FAMILY, footer_text_color=FOOTER_TEXT_COLOR,
               footer_bg_color=FOOTER_BACKGROUND_COLOR, footer_left_margin=FOOTER_LEFT_MARGIN,
               left_margin=DEFAULT_LEFT_MARGIN, right_margin=DEFAULT_RIGHT_MARGIN):
  """
  Create a poster from passport cover images
  
  Args:
      images_per_row: Number of passport images per row
      image_paths: List of file paths to passport images
      add_labels: Whether to add country name labels below each passport (default: True)
      font_size: Font size for country labels (default: 40)
      font_family: Font family for country labels (default: "Arial")
      text_color: RGBA tuple for text color (default: black)
      bg_color: RGBA tuple for text label background color (default: white)
      title: Optional title text to display at the top of the poster (default: None)
      title_height: Height of the title row in pixels (default: 200)
      title_font_size: Font size for the title (default: 120)
      title_font_family: Font family for the title (default: "Helvetica-Bold")
      title_text_color: RGBA tuple for title text color (default: black)
      title_bg_color: RGBA tuple for title background color (default: white)
      horizontal_spacing: Horizontal spacing in pixels between passport images (default: 0)
      vertical_spacing: Vertical spacing in pixels between rows (default: 0)
      background_color: RGBA tuple for overall poster background color (default: very light gray)
      footer_text: Optional footer text to display at the bottom of the poster (e.g., copyright)
      footer_height: Height of the footer row in pixels (default: 100)
      footer_font_size: Font size for the footer (default: 30)
      footer_font_family: Font family for the footer (default: "Arial")
      footer_text_color: RGBA tuple for footer text color (default: black)
      footer_bg_color: RGBA tuple for footer background color (default: white)
      footer_left_margin: Left margin for footer text in pixels (default: 40)
      left_margin: Left margin of the poster in pixels (default: 5)
      right_margin: Right margin of the poster in pixels (default: 5)
  
  Returns:
      PIL Image object containing the assembled poster
  """
  print(f"Creating poster with {len(image_paths)} pp covers")
  num_rows = len(image_paths) / images_per_row
  print(f"Aspect ratio will be {(2*images_per_row)/(3*num_rows)}")
  
  row_images = []
  elements_in_row_so_far = 0
  current_row_image = None
  
  for image_path in image_paths:
    country_name = get_country_name_from_path(image_path)
    
    # Special handling for countries with top/bottom bars
    cut_top_and_bottom_bar = country_name in set([
      "Argentina","Australia","Chile","Bulgaria","Bosnia+Herzegovina","Bolivia","China","Germany", "Guyana"
    ])
    
    # Get the processed passport image
    processed_image = get_processed_image_from_path(image_path, True)
    
    # Add text label if requested
    if add_labels:
      formatted_name = format_country_name(country_name)
      processed_image = add_text_label_to_image(
        processed_image, 
        formatted_name,
        font_size=font_size,
        font_family=font_family,
        text_color=text_color,
        bg_color=background_color
      )
    
    # Build rows horizontally
    if current_row_image is None:
      current_row_image = processed_image
      elements_in_row_so_far = 1
    elif elements_in_row_so_far < images_per_row:
      current_row_image = merge_horizontally(current_row_image, processed_image, horizontal_spacing, background_color)
      elements_in_row_so_far += 1
    else:
      # Start new row
      print(f"Adding new row to list")
      row_images.append(current_row_image)
      current_row_image = processed_image
      elements_in_row_so_far = 1
  
  # Add the last row if it exists
  if current_row_image is not None:
    row_images.append(current_row_image)

  # Merge all rows vertically
  poster = None
  print(f"There are {len(row_images)} rows to merge")
  for row_image in row_images:
    if poster is None:
      poster = row_image
    else:
      poster = merge_vertically(poster, row_image, vertical_spacing, background_color)
  
  # Add title row at the top if title is provided
  if title:
    print(f"Adding title: '{title}'")
    title_row = create_title_row(
      width=poster.size[0],
      title=title,
      title_height=title_height,
      title_font_size=title_font_size,
      title_font_family=title_font_family,
      title_text_color=title_text_color,
      title_bg_color=title_bg_color
    )
    # Add title row at the top
    poster = merge_vertically(title_row, poster, bg_color=background_color)
  
  # Add footer row at the bottom if footer_text is provided
  if footer_text:
    print(f"Adding footer: '{footer_text}'")
    footer_row = create_footer_row(
      width=poster.size[0],
      footer_text=footer_text,
      footer_height=footer_height,
      footer_font_size=footer_font_size,
      footer_font_family=footer_font_family,
      footer_text_color=footer_text_color,
      footer_bg_color=footer_bg_color,
      footer_left_margin=footer_left_margin
    )
    # Add footer row at the bottom
    poster = merge_vertically(poster, footer_row, bg_color=background_color)
  
  # Add left and right margins as the final step
  if left_margin > 0 or right_margin > 0:
    poster_width, poster_height = poster.size
    new_width = poster_width + left_margin + right_margin
    
    # Create new image with margins
    final_poster = Image.new("RGBA", (new_width, poster_height), background_color)
    
    # Paste the poster with left margin offset
    if poster.mode == 'RGBA':
      final_poster.paste(poster, (left_margin, 0), poster)
    else:
      final_poster.paste(poster, (left_margin, 0))
    
    poster = final_poster
  
  return poster

In [ ]:
def get_inner_most_path(absolute_path):
  innermost_path = absolute_path.split("/")[-1]
  return innermost_path

# for col_num in range(9,21):
#   poster = get_poster(col_num,sorted(all_pp_paths,key=get_inner_most_path))
#   poster.show()  

[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  Non

In [ ]:
poster = get_poster(23,sorted(all_pp_paths,key=get_inner_most_path))
poster.show()  

Creating poster with 199 pp covers
Aspect ratio will be 1.7721943048576212
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
There are 9 rows to merge


In [9]:
# for col_num in range(2,5):
poster = get_poster(10,sorted(region_to_full_pp_cover_paths["south_america"]))
  # poster.show()  

Creating poster with 13 pp covers
Aspect ratio will be 5.128205128205128
Adding new row to list
There are 2 rows to merge


In [10]:
# for col_num in range(5,11):
#   poster = get_poster(col_num,sorted(region_to_full_pp_cover_paths["north_america"]))
#   poster.show()  

In [11]:
# poster = get_poster(14,sorted(region_to_full_pp_cover_paths["north_america"]))
# poster.show()


In [12]:
# for col_num in range(5,11):
#   poster = get_poster(col_num,sorted(region_to_full_pp_cover_paths["asia"]))
#   poster.show()  

In [13]:
# for col_num in range(5,11):
#   poster = get_poster(col_num,sorted(region_to_full_pp_cover_paths["africa"]))
#   poster.show()  

To figure out next:
1. Passport shape cutout for all images or crop for all
2. Go to market

## Examples: Creating Posters with Country Labels, Titles, Spacing, and Footer

The updated `get_poster()` function now supports adding country name labels below each passport cover, an optional title at the top, horizontal and vertical spacing, and an optional footer at the bottom, all with customizable fonts and sizes.

### Parameters:
- `images_per_row`: Number of passports per row
- `image_paths`: List of passport image file paths
- `add_labels`: Whether to add country names (default: True)
- `font_size`: Size of the text (default: 40)
- `font_family`: Font name (default: "Arial")
- `text_color`: RGBA tuple for text color (default: black)
- `bg_color`: RGBA tuple for background color (default: white)
- `title`: Optional title text at the top (default: None)
- `title_height`: Height of the title row in pixels (default: 200)
- `title_font_size`: Font size for the title (default: 120)
- `title_font_family`: Font family for the title (default: "Helvetica-Bold")
- `title_text_color`: RGBA tuple for title text color (default: black)
- `title_bg_color`: RGBA tuple for title background color (default: white)
- `horizontal_spacing`: Horizontal spacing in pixels between passport images (default: 0)
- `vertical_spacing`: Vertical spacing in pixels between rows (default: 0)
- `footer_text`: Optional footer text at the bottom (e.g., copyright notice) (default: None)
- `footer_height`: Height of the footer row in pixels (default: 100)
- `footer_font_size`: Font size for the footer (default: 30)
- `footer_font_family`: Font family for the footer (default: "Arial")
- `footer_text_color`: RGBA tuple for footer text color (default: black)
- `footer_bg_color`: RGBA tuple for footer background color (default: white)
- `footer_left_margin`: Left margin for footer text in pixels (default: 40)

### Available System Fonts (macOS):
Common options: "Arial", "Helvetica", "Times New Roman", "Courier", "Georgia", "Verdana", "Helvetica-Bold"

### Optimized Grids for Standard Print Sizes:
- **23×10 grid**: Perfect for 36"×24" landscape posters (1.5:1 ratio)
- **15×14 grid**: Good for 24"×36" portrait posters (0.67:1 ratio)
- **12×17 grid**: Alternative for 24"×36" / 36"×48" portrait posters

In [ ]:
# Example 1: Create Asia poster with default settings (labels enabled, 40pt Arial font)
all_pp_paths = []
[[all_pp_paths.append(country) for country in countries_per_region] 
 for countries_per_region in list(region_to_full_pp_cover_paths.values())]

def get_inner_most_path(absolute_path):
    innermost_path = absolute_path.split("/")[-1]
    return innermost_path

# Asia poster - 7x7 grid (perfect square for 49 countries)
asia_poster = get_poster(
    images_per_row=7,
    image_paths=sorted(region_to_full_pp_cover_paths["asia"]),
    add_labels=True,
    font_size=40,
    font_family="Arial"
)
# asia_poster.show()
print(f"Asia poster size: {asia_poster.size}")

In [ ]:
# Example 2: World poster with larger font (for better readability)
world_poster_large_text = get_poster(
    images_per_row=20,
    image_paths=sorted(all_pp_paths, key=get_inner_most_path),
    add_labels=True,
    font_size=50,  # Larger font
    font_family="Helvetica"
)
# world_poster_large_text.show()
print(f"World poster size: {world_poster_large_text.size}")

In [ ]:
# Example 3: South America poster with smaller font
south_america_poster = get_poster(
    images_per_row=5,
    image_paths=sorted(region_to_full_pp_cover_paths["south_america"]),
    add_labels=True,
    font_size=30,  # Smaller font
    font_family="Arial"
)
# south_america_poster.show()

# Save the poster as JPG (high quality for printing)
# south_america_poster.convert("RGB").save("./produtti/south_america_with_labels.jpg", "JPEG", quality=95)
print(f"South America poster size: {south_america_poster.size}")

In [ ]:
# Example 4: Poster without labels (original behavior)
africa_no_labels = get_poster(
    images_per_row=8,
    image_paths=sorted(region_to_full_pp_cover_paths["africa"]),
    add_labels=False  # Disable labels
)
# africa_no_labels.show()
print(f"Africa poster (no labels) size: {africa_no_labels.size}")

# Example 5: Custom text and background colors
# Create a poster with dark text on light gray background
europe_custom_colors = get_poster(
    images_per_row=8,
    image_paths=sorted(region_to_full_pp_cover_paths["europe"]),
    add_labels=True,
    font_size=35,
    font_family="Helvetica",
    text_color=(50, 50, 50, 255),      # Dark gray text
    bg_color=(240, 240, 240, 255)      # Light gray background
)
# europe_custom_colors.show()
print(f"Europe poster (custom colors) size: {europe_custom_colors.size}")

In [ ]:
# Example 6: World poster with title
world_poster_with_title = get_poster(
    images_per_row=20,
    image_paths=sorted(all_pp_paths, key=get_inner_most_path),
    add_labels=True,
    font_size=45,
    font_family="Arial",
    title="Passports of the World",
    title_height=200,
    title_font_size=120,
    title_font_family="Helvetica-Bold"
)
# world_poster_with_title.show()

# Save the poster
# world_poster_with_title.convert("RGB").save("./produtti/world_poster_with_title.jpg", "JPEG", quality=95)
print(f"World poster with title size: {world_poster_with_title.size}")

In [ ]:
# Example 7: Oceania poster with horizontal spacing between passports
oceania_with_spacing = get_poster(
    images_per_row=4,
    image_paths=sorted(region_to_full_pp_cover_paths["oceania"]),
    add_labels=True,
    font_size=35,
    font_family="Arial",
    horizontal_spacing=30  # 30 pixels between each passport
)
# oceania_with_spacing.show()

# Compare sizes (without and with spacing):
# Without spacing: 4 passports × 675px = 2700px wide
# With spacing: 4 × 675 + 3 gaps × 30 = 2700 + 90 = 2790px wide
print(f"Oceania poster with 30px spacing size: {oceania_with_spacing.size}")

In [ ]:
# Example 8: World poster with title and footer (copyright notice)
world_poster_complete = get_poster(
    images_per_row=20,
    image_paths=sorted(all_pp_paths, key=get_inner_most_path),
    add_labels=True,
    font_size=45,
    font_family="Arial",
    title="Passports of the World",
    title_height=200,
    title_font_size=120,
    title_font_family="Helvetica-Bold",
    footer_text="© 2026 Your Name. All rights reserved.",
    footer_height=100,
    footer_font_size=30,
    footer_font_family="Arial"
)
# world_poster_complete.show()

# Save the poster
# world_poster_complete.convert("RGB").save("./produtti/world_poster_complete.jpg", "JPEG", quality=95)
print(f"World poster with title and footer size: {world_poster_complete.size}")

In [ ]:
# Example 10: World poster in 23×10 grid for 36"×24" landscape printing
world_poster_23x10 = get_poster(
    images_per_row=23,
    image_paths=sorted(all_pp_paths, key=get_inner_most_path),
    add_labels=True,
    font_size=45,
    font_family="Arial",
    horizontal_spacing=30,
    vertical_spacing=8,  # Reduced vertical spacing for landscape
    title="Passports of the World",
    title_height=250,
    title_font_size=220,
    title_font_family="Helvetica-Bold",
    footer_text="© 2026. All rights reserved.",
    footer_height=60,
    footer_font_size=26,
    footer_font_family="Arial"
)
# world_poster_23x10.show()

# This creates a landscape orientation poster perfect for 36"×24" printing
# 23 wide × 10 tall = 230 slots (199 countries + 31 empty spots)
# Target ratio: 1.5:1 (3:2 ratio for standard landscape prints)
# Save the poster
# world_poster_23x10.convert("RGB").save("./produtti/world_poster_23x10_landscape.jpg", "JPEG", quality=95)
print(f"23×10 grid poster size: {world_poster_23x10.size}")
print(f"Aspect ratio: {world_poster_23x10.size[0] / world_poster_23x10.size[1]:.3f}:1")
print(f"Perfect for 36\"×24\" landscape printing!")

In [ ]:
# Example 9: World poster in 12×17 grid for standard print sizes (24"×36" / 36"×48")
world_poster_12x17 = get_poster(
    images_per_row=12,
    image_paths=sorted(all_pp_paths, key=get_inner_most_path),
    add_labels=True,
    font_size=50,
    font_family="Arial",
    horizontal_spacing=30,
    vertical_spacing=15,  # Add vertical spacing between rows
    title="Passports of the World",
    title_height=400,
    title_font_size=320,
    title_font_family="Helvetica-Bold",
    footer_text="© 2026. All rights reserved.",
    footer_height=80,
    footer_font_size=28,
    footer_font_family="Arial"
)
# world_poster_12x17.show()

# This creates a portrait orientation poster perfect for standard print sizes
# 12 wide × 17 tall = 204 slots (199 countries + 5 empty spots)
# Save the poster
# world_poster_12x17.convert("RGB").save("./produtti/world_poster_12x17.jpg", "JPEG", quality=95)
print(f"12×17 grid poster size: {world_poster_12x17.size}")
print(f"Aspect ratio: {world_poster_12x17.size[0] / world_poster_12x17.size[1]:.2f}:1")
print(f"Perfect for 24\"×36\" or 36\"×48\" portrait printing!")